In [197]:
import pandas as pd
import numpy as np

import spacy

from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter

from heapq import nlargest

from z_functions import extract

In [138]:
nlp = spacy.load("en_core_web_lg")

In [139]:
movies = pd.read_csv('../data/genre_groups/master_genre_df.csv')
movies.head(2)

,imdb_id,title,overviews,join_director,join_cast,vote_average,vote_count,genre,user_plots,synopsis,...,Fantasy,Mystery,Thriller,Western,Family,Sci-Fi,Romance,Film-Noir,Musical,Music
0,tt0111161,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,FrankDarabont,TimRobbins MorganFreeman BobGunton,8.7,18845,Drama,['Two imprisoned men bond over a number of yea...,"In 1947, Andy Dufresne (Tim Robbins), a banker...",...,0,0,0,0,0,0,0,0,0,0
1,tt0068646,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",FrancisFordCoppola,AlPacino MarlonBrando JamesCaan,8.7,14225,Crime Drama,"[""An organized crime dynasty's aging patriarch...","In late summer 1945, guests are gathered for t...",...,0,0,0,0,0,0,0,0,0,0


In [179]:
doc = nlp(movies['user_plots'][21])

In [180]:
len(list(doc.sents))

21

In [181]:
keywords = []
pos_tag = ['NOUN', 'VERB']

#filter tokens
for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keywords.append(token.text)

In [182]:
freq_word = Counter(keywords)
freq_word.most_common(n=20)

[('brothers', 6),
 ('killed', 6),
 ('brother', 6),
 ('find', 6),
 ('invasion', 5),
 ('mother', 4),
 ('bring', 4),
 ('mission', 4),
 ('action', 3),
 ('members', 3),
 ('learns', 3),
 ('men', 3),
 ('unit', 3),
 ('war', 3),
 ('Following', 2),
 ('fight', 2),
 ('secure', 2),
 ('beachhead', 2),
 ('fighting', 2),
 ('receive', 2)]

In [183]:
max_freq = Counter(keywords).most_common()[0][1]

#normalizing 
for word in freq_word.keys():
    freq_word[word] = (freq_word[word] / max_freq)
freq_word.most_common(5)

[('brothers', 1.0),
 ('killed', 1.0),
 ('brother', 1.0),
 ('find', 1.0),
 ('invasion', 0.8333333333333334)]

In [184]:
sent_strength = {}

#loop through sentences
for sent in doc.sents:
    
    #loop through words in sentence
    for word in sent:
        
        #checking if word is in our frequent_words dict.
        if word.text in freq_word.keys():
            
            #adding freq_word value to sent value
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_word[word.text]
            
            else:
                sent_strength[sent] = freq_word[word.text]

In [190]:
summarized_sentences = nlargest(5, sent_strength, key=sent_strength.get)

In [191]:
final_sentences = [w.text for w in summarized_sentences]

In [192]:
final_summary = ' '.join(final_sentences)

In [193]:
final_summary

'The United States Army Chief of Staff, George C. Marshall, is given an opportunity to alleviate some of her grief when he learns of a fourth brother, Private James Ryan, and decides to send out 8 men (Cpt. Miller and select members from 2nd Rangers) to find him and bring him back home to his mother...\', "After the invasion of fortress Europe on June 6th 1944, Cpt. Miller leads his squad from the 2nd Ranger Battalion of the 29th Infantry Division, on a mission to find and bring home Private James Francis Ryan after the death of his brothers. This exciting war thriller brings the reality of history\'s bloodiest war into the homes of ordinary people, but also brings to light the reality of broken and lost families in a time of total and encompassing war.", \'The film opens with the Allied invasion on the Normandy beach on June 6, 1944. [\'Following the Normandy Landings, a group of U.S. soldiers go behind enemy lines to retrieve a paratrooper whose brothers have been killed in action.\'

In [195]:
extract(nlp(final_summary))

'find bring invasion war learns brother members mother mission brothers '